<a href="https://colab.research.google.com/github/jibarletta/PAECIS/blob/main/C%C3%B3digo_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Instalación de paquetes necesarios para los procedimientos de extracción, preprocesamiento y procesamiento#

Fase 1: Extracción

In [ ]:
#A continuación se genera un datastamp para su posterior uso. Será muy necesario para clasificar datos y archivos#
filename_date = dt.now().strftime('%Y-%m-%d')  # -%H-%M removido del filename

In [ ]:
import pandas as pd
import datetime
from datetime import datetime as dt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
!pip install bertopic
import re
import unicodedata
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
!python -m spacy download es_core_news_sm
nlp = spacy.load("es_core_news_sm")
import numpy
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim import corpora
from bertopic import BERTopic
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models import CoherenceModel
from sklearn.metrics import silhouette_score

In [ ]:
#Se convierte el diccionario en un dataframe. Se lo guarda en CVS. Luego se lo formatea para concatenarlo con los posteriores datos que se sacarán de Reedit. También se arma un CSV aparte solo con los 10 temas del día#
temas = pd.DataFrame(post_dict)
temas.to_csv(f"{Path.cwd()}/Bases/Temas/Reddit_Arg_Temas{filename_date}.csv")
dataset = pd.read_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv", index_col=0)
nuevods = pd.concat([dataset, temas], sort=False)
nuevods.reset_index(inplace=True, drop=True)
nuevods.to_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv")

In [ ]:
#Se realiza el mismo procedimiento para los comentarios de cada uno de los post del diccionario anterior. Se concatena al dataset general
lista_ids = temas["id"]
for postid in lista_ids:
    comment_dict = {"id": [],
                    "context": [],
                    "text": [],
                    "upv": [],
                    "author": [],
                    "date": [],
                    "type": [],
                    }
    posteo = reddit.submission(postid)
    posteo.comments.replace_more(limit=0)
    for comentario in posteo.comments.list():
        if comentario.author == 'empleadoEstatalBot':  # Elimina el bot que postea la noticia en caso de existir.
            continue
        comment_dict["id"].append(comentario.id)
        p_split = comentario.parent_id.split('_')
        idctx = p_split[1]
        comment_dict['context'].append(idctx)
        comment_dict["text"].append(comentario.body)
        comment_dict["upv"].append(comentario.score)
        comment_dict["author"].append(comentario.author)
        comment_dict['date'].append(filename_date)
        comment_dict['type'].append('comment')

In [ ]:
#En esta última sección se termina de concatenar el dataset final y se separa aparte un archivo sólo con los comentarios del correspondiente post
    dataset = pd.read_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv", index_col=0)
    comms = pd.DataFrame(comment_dict)
    nuevods = pd.concat([dataset, comms], sort=False)
    nuevods.reset_index(inplace=True, drop=True)
    nuevods.to_csv(f"{Path.cwd()}/Bases/Dataset/Red_Arg_Dataset.csv")
    comms.to_csv(f"{Path.cwd()}/Bases/Comentarios/Red_Arg_Comm_Post-{postid}-{filename_date}.csv")
    archivo = f"Red_Arg_Comm_Post-{postid}-{filename_date}.csv"

Fase #2: Preprocesamiento

In [ ]:
# Se define el path del archivo CVS que contiene todo el Dataset unificado y se carga el CSV en un Dataframe
file_path = 'https://raw.githubusercontent.com/luchamaria13/TIF2/refs/heads/main/Red_Arg_Dataset%20-%20Red_Arg_Dataset.csv.csv'

try:
    df = pd.read_csv(file_path)
    print("Archivo cargado exitosamente. Muestra de datos:")
    print(df.head())
except Exception as e:
    print(f"Error al cargar el archivo: {e}")

Fase #3: Procesamiento

3.1. Identificación de post más votados y más comentados
Pese a que este no es el primer objetivo específico, teniendo en cuenta que es el procedimiento con menor nivel de complejidad es el que se lleva a cabo primero

In [ ]:
# Función para limpiar texto
def limpiar_texto(texto):
    if not isinstance(texto, str) or not texto.strip():
        return ''
    texto = texto.lower()  # Convertir a minúsculas
    texto = re.sub(r'[^\w\s]', '', texto)  # Quitar caracteres especiales
    texto = re.sub(r'\d+', '', texto)  # Quitar números
    texto = ' '.join(texto.split())  # Eliminar espacios extras
    #texto = eliminar_tildes(texto)  # Eliminar tilde #
    if len(texto.split()) < 4:  # Ignorar textos demasiado cortos
        return ''

    # Lista de stopwords
    stop_words = set(stopwords.words('spanish'))
    stop_words.update([
        "acá","link","jar","jajaj", "jajajaj", "jajajajajar", "jajajajaja", "kjjj", "jajaja", "jajajja", "kjj",
        "eo", "eto", "eta", "u", "che", "reddit", "the", "only", "get", "meta", "by", "removed",
        "ear", "ete", "jajajajajajaja", "que", "de", "el", "por", "se", "está", "lo", "en",
        "si", "https", "preview", "redd", "it", "giphy", "gif", "jpeg", "webp", "www", "png",
        "format", "com", "width", "y", "pero", "como", "solo", "ma", "sino", "q", "ahi", "jajaja",
        "jaja", "hacer", "mas", "porque", "muy", "donde", "siempre", "ya", "uno", "hace","the","one","atchv","youtube","xd","jajs","username","usuario","oh","ho","di","etc","xb"
    ])

    # Eliminar stopwords del texto
    palabras = [palabra for palabra in texto.split() if palabra not in stop_words]
    return " ".join(palabras)

In [ ]:
#En primer lugar se crea una función para analizar los upvotes y downvotes de los post y los comentarios del dataset
def analizar_upvotes(df):
    # Análisis estadístico descriptivo de upvotes
    resumen_upvotes = df['upv'].describe()

    # Distribución de upvotes por rango
    df['rango_upvotes'] = pd.cut(df['upv'],
        bins=[-float('inf'), 0, 10, 50, 100, 500, float('inf')],
        labels=['Negativos', '1-10', '11-50', '51-100', '101-500', '500+'])

    # Conteo de posts por rango de upvotes
    distribucion_upvotes = df['rango_upvotes'].value_counts().sort_index()

    # Porcentaje de posts por rango de upvotes
    porcentaje_upvotes = (distribucion_upvotes / len(df) * 100).round(2)

    # Top 10 posts con más upvotes
    top_10_upvotes = df.nlargest(10, 'upv')[['upv', 'text']]

    # Visualizaciones
    plt.figure(figsize=(15,10))

    # Subplot 1: Distribución de Upvotes
    plt.subplot(2, 2, 1)
    sns.barplot(x=distribucion_upvotes.index, y=distribucion_upvotes.values)
    plt.title('Distribución de Upvotes')
    plt.xlabel('Rango de Upvotes')
    plt.ylabel('Número de Posts')
    plt.xticks(rotation=45)

    # Subplot 2: Porcentaje de Upvotes
    plt.subplot(2, 2, 2)
    sns.barplot(x=porcentaje_upvotes.index, y=porcentaje_upvotes.values)
    plt.title('Porcentaje de Posts por Rango de Upvotes')
    plt.xlabel('Rango de Upvotes')
    plt.ylabel('Porcentaje de Posts')
    plt.xticks(rotation=45)


    plt.tight_layout()
    plt.show()

    # Gráfico de Top 10 posts con más upvotes
    plt.figure(figsize=(14,7))
    bars = plt.barh(
    top_10_upvotes['text'].str[:50],
    top_10_upvotes['upv'],
    color='skyblue'
)
    plt.title('Top 10 Posts con Más Upvotes')
    plt.xlabel('Número de upvotes')
    plt.ylabel('Texto del post')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()



# Ejemplo de uso
resultados_upvotes = analizar_upvotes(df)

# Filtrar posts con votos negativos y ordenarlos
# Filtrar posts con votos negativos y ordenarlos
top_10_downvotes = df[df['upv'] < 0].nsmallest(10, 'upv')

plt.figure(figsize=(16,8))
bars = plt.barh(
    top_10_downvotes['text'].str[:50],
    abs(top_10_downvotes['upv']),  # Valor absoluto para longitud de barra
    color='salmon'
)
plt.title('Top 10 Posts con Más Downvotes en Reddit Argentina', fontsize=15)
plt.xlabel('Número de Downvotes', fontsize=12)
plt.ylabel('Extracto del Texto', fontsize=12)

# Añadir etiquetas con el número de votos al final de cada barra
for bar in bars:
    width = bar.get_width()
    plt.text(
        width,
        bar.get_y() + bar.get_height()/2.,
        f' {int(width):,} downvotes',
        va='center',
        ha='left',
        fontsize=10,
        color='darkred'
    )

plt.tight_layout()
plt.show()

In [ ]:
# Filtrar solo los comentarios
comentarios = df[df['type'].str.strip().str.lower() == 'comment']

# Contar el número de comentarios por cada post (usando context)
conteo_comentarios = comentarios.groupby('context').size().reset_index(name='n_comentarios')

# Ordenar por número de comentarios en orden descendente
conteo_comentarios = conteo_comentarios.sort_values(by='n_comentarios', ascending=False)

# Seleccionar los 10 posts con más comentarios
top_10_posts = conteo_comentarios.head(10)

# Filtrar los posts principales (type == 'post')
posts = df[df['type'].str.strip().str.lower() == 'post']

# Combinar la información de los textos con el conteo de comentarios
tabla_completa = top_10_posts.merge(posts, left_on='context', right_on='id')

# Seleccionar columnas relevantes: ID del post, texto y número de comentarios
tabla_completa = tabla_completa[['id', 'text', 'n_comentarios']]

# Mostrar la tabla
print(tabla_completa)
# Guardar la tabla en un archivo CSV
tabla_completa.to_csv("top_10_posts.csv", index=False, encoding='utf-8-sig')

# Gráfica de los 10 posts más comentados
plt.figure(figsize=(10, 6))
plt.bar(tabla_completa['id'].astype(str), tabla_completa['n_comentarios'], color='skyblue')
plt.title('Top 10 Posts con Más Comentarios', fontsize=14)
plt.xlabel('ID del Post', fontsize=12)
plt.ylabel('Número de Comentarios', fontsize=12)
plt.xticks(rotation=45, fontsize=10)
plt.tight_layout()
plt.show()

3.2. Identificación de los elementos del discurso libertario en el corpus.
La estrategia que se utilizó en este caso fue el uso de bag words. Así, se construyó un dicionario de palabras presentes en el dataset que hiciera parte de las categorias que componen el discurso libertario (identificadas en la revisión de la literatura)

In [ ]:
# Definir palabras clave para cada categoría
palabras_clave = {
    "Posturas en contra de la casta política": [
        "CASTA", "CASTAA", "Ks CASTA", "KUKAS CASTA", "CASTA ZURDA",
        "CASTA MAFIOSA", "VERDADERA CASTA", "PAGA LA CASTA",
        "CASTA DE LADRONES", "PERONISMO CAMUFLADO CASTA", "LOS K CASTA",
        "TIEMBLA LA CASTA", "CASTA MUCHACHES PERONISTES",
        "CASTA VIOLADORES CORRUPTOS", "BENEFICIO DE CASTA"
    ],
    "Rechazo anti kirchnerismo-peronismo-izquierdas": [
        "K", "PERIODISTAS K", "PERONISTA ASOCIACIÓN ILÍCITA",
        "K CHORROS", "IZQUIERDA TERRORISTA ANTIDEMOCRÁTICA",
        "PERONISTA TOMA PODER", "PERONISTAS IZQUIERDA GOLPISTA",
        "NO SOY K PERO", "PERONCHO MEDIOS K", "LOS KA PERONISTA",
        "LOS K", "K SOCIALISTAS", "IZQUIERDA GOLPISTA", "KAKA",
        "KUKA", "KUKAS", "PRE-KUKAS KIRCHO", "KIRCHOS", "KIRCHNERATO",
        "KUKARACHA", "KUKARACHOS", "KUKARDO", "KUKARDOS", "KUKALOIDE",
        "KUKANETA", "KUKALISMO", "KUKAMONGA", "KUKATRILES",
        "INKUKAY", "KUKATRAP", "PERONCHO", "PERONCHOS", "CAMARRRADA",
        "ZURDO", "ZURDOS", "ZURDAJE", "SURDITO", "NSKP", "MONTONEROS",
        "GUERRILLEROS", "IZQUIERDA", "PERONISTA", "PERONISTAS"
    ],
    "Anticolectivismo/Racismo/Xenofobia": [
        "GORDOS PIQUETEROS", "PERUANOS COME PALOMAS",
        "BOLIVIANOS PARAGUAYOS DEPREDADORES", "SIMIOS DEL CONGOURBANO",
        "PLANES KIRCHERISMO PERONISMO", "ÑOQUIS SIMIOS TIRAPIEDRAS",
        "PERUANO OTAKU", "PLANERO", "MAPUTRUCHOS", "MAPUCHES",
        "PERUANOS", "PERUANO", "COMECHINGONES", "MACACOS", "CONGOURBANO"
    ],
    "Antifeminismo/anti población LGTBI": [
        "PIBARDOS LGVT", "TRAVA", "FEMINAZIS", "GAY", "TROLO",
        "PUTO", "TORTA", "PIBIS"
    ],
    "A favor de la defensa de ley y orden": [
        "PENA", "PENAS", "REPRESIÓN", "DELITO ADULTO PENA ADULTO",
        "REPRESIÓN ORDEN PÚBLICO", "PENAS SON CHISTE",
        "VERDADERA CORRECTA REPRESIÓN"
    ]
}

# El resto del código permanece igual que en la versión anterior
# Inicializar un contador para almacenar las frecuencias por categoría
frecuencias = {categoria: 0 for categoria in palabras_clave.keys()}

# Contar la frecuencia de palabras clave en cada categoría
for categoria, palabras in palabras_clave.items():
    # Crear un patrón regex para las palabras clave sin grupos de captura
    patron = r'\b(?:' + '|'.join(palabras) + r')\b'
    # Sumar la cantidad de coincidencias en todo el dataset
    frecuencias[categoria] = df['text'].str.count(patron).sum()

# Mostrar frecuencias por categoría
print("Frecuencias por categoría:")
for categoria, frecuencia in frecuencias.items():
    print(f"{categoria}: {frecuencia}")

# Visualizar las frecuencias en un gráfico de barras
plt.figure(figsize=(10, 6))
plt.bar(frecuencias.keys(), frecuencias.values(), color='skyblue')
plt.title('Frecuencia de palabras clave por categoría', fontsize=14)
plt.xlabel('Categorías', fontsize=12)
plt.ylabel('Frecuencia', fontsize=12)
plt.xticks(rotation=45, fontsize=10, ha='right')
plt.tight_layout()
plt.show()

# Análisis detallado por texto: agregar columnas para cada categoría
for categoria, palabras in palabras_clave.items():
    patron = r'\b(?:' + '|'.join(palabras) + r')\b'
    # Crear una columna booleana que indica si un texto contiene palabras clave de la categoría
    df[categoria] = df['text'].str.contains(patron, regex=True)

# Contar la frecuencia de palabras específicas de cada categoría en todo el dataset
for categoria, palabras in palabras_clave.items():
    word_counts = Counter()

    # Contar cada palabra de la categoría en los textos
    for palabra in palabras:
        word_counts[palabra] = df['text'].str.count(palabra).sum()

    # Mostrar las 10 palabras más comunes
    top_words = word_counts.most_common(10)

    # Crear un gráfico de barras para las 10 palabras más frecuentes
    if top_words:
        palabras, frecuencias = zip(*top_words)

        plt.figure(figsize=(10, 6))
        plt.bar(palabras, frecuencias, color='skyblue')
        plt.title(f'Las 10 palabras más frecuentes en la categoría {categoria}', fontsize=14)
        plt.xlabel('Palabras', fontsize=12)
        plt.ylabel('Frecuencia', fontsize=12)
        plt.xticks(rotation=45, fontsize=10, ha='right')
        plt.tight_layout()
        plt.show()

# Exportar el dataset con las nuevas columnas
df.to_csv("analisis_categorias.csv", index=False, encoding='utf-8-sig')

print("El análisis detallado se ha exportado como 'analisis_categorias.csv'.")



3.3. Identificación de los tópicos presentes en el dataset
Para esto se utilizó una estrategia de topic modeling.

In [ ]:
#Primero armamos un diccionario con los 10 post más populares del subrredit Argentina#
post_dict = {"id": [],
             "context": [],
             "text": [],
             "upv": [],
             "author": [],
             "date": [],
             "type": [],
             }
arg_subreddit = reddit.subreddit('Argentina')
for post in arg_subreddit.top(limit=10, time_filter='day'):
    post_dict["id"].append(post.id)
    post_dict["context"].append(post.id)
    post_dict["text"].append(post.title)
    post_dict["upv"].append(post.score)
    post_dict["author"].append(post.author)
    post_dict["date"].append(filename_date)
    post_dict['type'].append('post')

In [ ]:
#Definición de una muestra de los textos lematizados para probar el modelo#
sample_df = df.sample(frac=1, random_state=42)
documentos = sample_df['text_lematizado'].tolist()

In [ ]:
#Crear diccionario y corpus
dictionary = corpora.Dictionary([doc.split() for doc in sample_df['text_lematizado']])
corpus = [dictionary.doc2bow(doc.split()) for doc in sample_df['text_lematizado']]

In [ ]:
# Create the BERTopic model
topic_model = BERTopic(language="spanish", calculate_probabilities=True, verbose=True)
# Fit the model to your data
topics, probs = topic_model.fit_transform(sample_df['text_lematizado'].tolist())
topic_info = topic_model.get_topic_info()
topic_info_df = topic_info.copy()
topic_info_df['Palabras_clave'] = topic_info_df['Topic'].apply(lambda topic_id: ', '.join([word for word, _ in topic_model.get_topic(topic_id)][:10]) if topic_id != -1 else '')

In [ ]:
# Función para lematizar texto
import re
def lematizar_texto(texto):
    doc = nlp(texto)
    tokens_lematizados = [token.lemma_ for token in doc if token.is_alpha]  # Solo palabras
    return " ".join(tokens_lematizados)

# Aplicar funciones de limpieza y lematización al dataset
if 'text' in df.columns:
    df['text_lematizado'] = df['text'].apply(limpiar_texto).apply(lematizar_texto)
    texto = df['text_lematizado'].tolist()  # Convertir columna en lista
    print("Texto procesado:")
    print(texto[:5])  # Muestra los primeros 5 textos procesados
    num_entradas = len(df[df['text_lematizado'] != ''])

In [ ]:
def analizar_temas_detallado(topic_model, n_temas=10):
    print("\nAnálisis detallado de los temas principales:")
    for topic_id in range(min(n_temas, len(topic_model.get_topic_info()))):
        if topic_id != -1:
            print(f"\nTema {topic_id}:")
            palabras = topic_model.get_topic(topic_id)
            print("Palabras clave:", ", ".join([word for word, _ in palabras[:10]]))
            docs = topic_model.get_representative_docs(topic_id)
            print("\nEjemplos de documentos:")
            for doc in docs[:3]:
                print(f"- {doc[:200]}...")
            size = topic_model.get_topic_sizes()[topic_id]
            print(f"\nCantidad de documentos en este tema: {size}")

In [ ]:
def visualizar_palabras_clave(topic_model, n_temas=10, n_palabras=10):
    import matplotlib.pyplot as plt
    from nltk.corpus import stopwords

    # Definir stopwords dentro de la función
    stop_words = set(stopwords.words('spanish'))
    stop_words.update([
        "jajaj", "jajajaj", "jajajajajar", "jajajajaja", "kjjj", "jajaja", "jajajja", "kjj",
        "eo", "eto", "eta", "u", "che", "reddit", "the", "only", "get", "meta", "by", "removed",
        "ear", "ete", "jajajajajajaja", "que", "de", "el", "por", "se", "está", "lo", "en",
        "si", "https", "preview", "redd", "it", "giphy", "gif", "jpeg", "webp", "www", "png",
        "format", "com", "width", "y", "pero", "como", "solo", "ma", "sino", "q", "ahi", "jajaja",
        "jaja", "hacer", "mas", "porque", "muy", "donde", "siempre", "ya", "uno", "hace","atchv","youtube","xd","jajs","username","usuario"
    ])


    fig, axes = plt.subplots(min(n_temas, 5), 2, figsize=(15, 4*min(n_temas, 5)))
    axes = axes.flatten()

    for idx, topic_id in enumerate(range(min(n_temas, len(topic_model.get_topic_info())))):
        if topic_id != -1 and idx < len(axes):
            palabras = topic_model.get_topic(topic_id)

            # Filtra las palabras clave aquí
            palabras_clave_filtradas = [(word, score) for word, score in palabras if word not in stop_words]

            # Usa las palabras clave filtradas para la visualización
            palabras = dict(palabras_clave_filtradas[:n_palabras])

            axes[idx].barh(list(palabras.keys()), list(palabras.values()))
            axes[idx].set_title(f'Tema {topic_id}')

    plt.tight_layout()
    plt.show()

In [ ]:
visualizar_palabras_clave(topic_model)


In [ ]:
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
from sklearn.metrics import silhouette_score
import numpy as np

# Calculate coherence using Gensim's CoherenceModel
topic_words = [topic_model.get_topic(topic_id) for topic_id in range(len(topic_model.get_topic_info())) if topic_id != -1]
topic_words = [topic for topic in topic_words if not isinstance(topic, bool)]
topic_words = [[word for word, _ in topic] for topic in topic_words]

# Create a Gensim dictionary
dictionary = Dictionary([doc.split() for doc in documentos])

# Convert topic words to token IDs
topic_words_ids = [[dictionary.token2id[word] for word in topic if word in dictionary.token2id] for topic in topic_words]

# Calculate the coherence score (C_v)
coherence_model = CoherenceModel(
    topics=topic_words_ids,
    texts=[doc.split() for doc in documentos],
    dictionary=dictionary,
    coherence='c_v'
)
coherence_score = coherence_model.get_coherence()
print(f"Coherence Score (C_v): {coherence_score}")




Coherence Score (C_v): 0.3316138385826245


In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def calcular_silhouette_score(documentos, topics):
    # Vectorización de los documentos
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(documentos)

    # Convertir matriz dispersa a densa
    X_dense = X.toarray()

    # Calcular Silhouette Score
    try:
        silhouette_avg = silhouette_score(X_dense, topics)
        return silhouette_avg
    except Exception as e:
        print(f"Error al calcular Silhouette Score: {e}")
        return None

# Calcular Silhouette Score para BERTopic
silhouette_bertopic = calcular_silhouette_score(documentos, topics)
print(f"Silhouette Score de BERTopic: {silhouette_bertopic}")

In [ ]:
fig_topics = topic_model.visualize_topics()
fig_topics.show()

fig_hierarchy = topic_model.visualize_hierarchy()
fig_hierarchy.show()


In [ ]:

def calcular_perplexity_bertopic(modelo_bertopic, topics):

    _, counts = np.unique(topics, return_counts=True)
    probabilidades = counts / len(topics)
    entropia = -np.sum(probabilidades * np.log2(probabilidades))


    perplexity = 2 ** entropia

    return perplexity

perplexity_bertopic = calcular_perplexity_bertopic(topic_model, topics)
print(f"Perplexity de BERTopic: {perplexity_bertopic}")